# Jane Street - GPU Submission
This is the actual notebook submitted to the competition. First, we load the trained models and use them to implement the `predict()` function.

In [ ]:
import json
import os

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

In [ ]:
# load the model
model_folder = os.path.join(os.pardir, "input", "jane-street-gpu")

with open(os.path.join(model_folder, "model.json"), "r") as f:
    model_json = f.read()

model = keras.models.model_from_json(model_json)
model.load_weights(os.path.join(model_folder, "model.h5"))

# compile the model
model.call = tf.function(model.call, experimental_relax_shapes=True)

# get the optimal threshold
utility_folder = os.path.join(os.pardir, "input", "jane-street-gpu-utility")

with open(os.path.join(utility_folder, "threshold.json")) as f:
    threshold_json = json.loads(f.read())

threshold = float(threshold_json["threshold"])

# get the medians to impute missing values
median = pd.read_csv(os.path.join(model_folder, "impute.csv"), index_col=0, squeeze=True)

In [ ]:
# function to return predictions for the current trade
def predict(sample):
    X = sample.to_numpy()
    nan = np.isnan(X)
    X[nan] = median[nan.squeeze()]
    prob = model(X, training=False).numpy().item()
    return 1 if prob > threshold else 0

Then we use the Jane Street API to make the predictions. In this setup the iterator `iter_test` yields single samples `test_df` in chronological order. The predict function needs to decide whether to accept or reject the trade at this time-point using only the current sample and the previous samples stored in `hist_df`. Set `pred_df` to 1 to accept the trade or to 0 to reject the trade.

In [ ]:
import janestreet
env = janestreet.make_env()
iter_test = env.iter_test()

# initialize things with first sample
sample, pred = next(iter_test)
features = [c for c in sample.columns if "feature" in c]
median = median[features].to_numpy()
pred.action = predict(sample[features])
env.predict(pred)

# loop over remaining samples
for (sample, pred) in iter_test:
    pred.action = predict(sample[features]) if sample["weight"].item() > 0.0 else 0
    env.predict(pred)